In [14]:
# Import necessary libraries
import fastf1 as ff1
import time
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt



year = 2024 # Year of  gran prinx
# Specify the grand prix you want to analyze
race = 'Monaco'  # Replace
# Specify the driver you want to analyze
target_driver = 'LEC'  # Replace with the driver's abbreviation (e.g., 'VER' for Max Verstappen)

# Load the session data for the specified race
session = ff1.get_session(year, race, 'R')  # Example: Monaco 2023 Race
session.load()

loc = session.event.Location

lunghezza_circuito = {
    'Melbourne': 5278,
    'Shanghai': 5451,
    'Suzuka': 5807,
    'Bahrain': 5412,
    'Jeddah': 6174,
    'Miami': 5412,
    'Imola': 4909,
    'Monaco': 3337,
    'Spagna': 4657,
    'Montreal': 4361,
    'Austria': 4318,
    'Silverstone': 5891,
    'Spa': 7004,
    'Hungaroring': 4381,
    'Zandvoort': 4259,
    'Monza': 5793,
    'Baku': 6003,
    'Singapore': 4940,
    'Austin': 5513,
    'Mexico City': 4304,
    'Interlagos': 4309,
    'Las Vegas': 6201,
    'Qatar': 5419,
    'Abu Dhabi': 5281,
    'Le Castellet': 5842,
    'Sochi': 5848,
    'Portimao': 4653,
    'Istanbul': 5338,
    'Nurburgring': 5148,
    'Hockenheim': 4574,
    
    'Adelaide': 3780,
    'Phoenix': 3770,
    'Estoril': 4415,
    'Magny-Cours': 4411,
    'Indianapolis': 4180,
    'Sepang': 5543,
    'Valencia': 4005,
    'Yeongam': 5603,
    'Jerez': 4423,
    'Kyalami': 4300,
    'Estoril': 4415,
    'Donington Park': 4023,
    'Fuji': 4571
}.get(race, 0) 

# Get the laps data for the target driver
driver_laps = session.laps.pick_driver(target_driver)


# devo prendere le lunghezze di tutti i circuiti
# ciclo for per ogni giro, si moltiplica il tire life per lunghezza circuito e si passa all'altro giro

#alla fine vorrei avere pilota, tempo giro, numero giro, tire life, metri gomma, tipo di mescola, is fesh tire


print(f'Location: {loc}')
print(f'Length of the circuit: {lunghezza_circuito}')
print(driver_laps)

# Save data to CSV files
#driver_laps.to_csv(f'T. {target_driver}_lap_data_in_{race}_year_{year}.csv', index=False)

core           INFO 	Loading data for Monaco Grand Prix - Race [v3.4.5]
req            INFO 	Using cached data for session_info
req            INFO 	Using cached data for driver_info
req            INFO 	Using cached data for session_status_data
req            INFO 	Using cached data for lap_count
req            INFO 	Using cached data for track_status_data
req            INFO 	Using cached data for _extended_timing_data
req            INFO 	Using cached data for timing_app_data
core           INFO 	Processing timing data...
req            INFO 	Using cached data for car_data
req            INFO 	Using cached data for position_data
req            INFO 	Using cached data for weather_data
req            INFO 	Using cached data for race_control_messages
core           INFO 	Finished loading data for 20 drivers: ['16', '81', '55', '4', '63', '1', '44', '22', '23', '10', '14', '3', '77', '18', '2', '24', '31', '11', '27', '20']


Location: Monaco
Length of the circuit: 3337
                     Time Driver DriverNumber                LapTime  \
0  0 days 01:36:02.379000    LEC           16 0 days 00:40:56.330000   
1  0 days 01:39:00.200000    LEC           16                    NaT   
2  0 days 01:40:24.824000    LEC           16 0 days 00:01:24.624000   
3  0 days 01:41:47.449000    LEC           16 0 days 00:01:22.625000   
4  0 days 01:43:09.244000    LEC           16 0 days 00:01:21.795000   
..                    ...    ...          ...                    ...   
73 0 days 03:13:12.803000    LEC           16 0 days 00:01:16.105000   
74 0 days 03:14:29.190000    LEC           16 0 days 00:01:16.387000   
75 0 days 03:15:45.998000    LEC           16 0 days 00:01:16.808000   
76 0 days 03:17:02.931000    LEC           16 0 days 00:01:16.933000   
77 0 days 03:18:21.567000    LEC           16 0 days 00:01:18.636000   

    LapNumber  Stint             PitOutTime              PitInTime  \
0         1.0    1.0

c:\Users\dilau\Desktop\TeamProjF1\FASTF1\.venv\Lib\site-packages\fastf1\core.py:3022: FutureWarning: pick_driver is deprecated and will be removed in a future release. Use pick_drivers instead.
  warnings.warn(("pick_driver is deprecated and will be removed"


In [19]:
# Import necessary libraries
import fastf1 as ff1
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

year = 2023  # Year of grand prix
race = 'Monaco'  # Replace with desired race
target_driver = 'LEC'  # Replace with the driver's abbreviation

# Load the session data for the specified race
session = ff1.get_session(year, race, 'R')  # Example: Monaco 2023 Race
session.load()

# Get circuit length (in meters)
circuit_length = {
    'Melbourne': 5278,
    'Shanghai': 5451,
    'Suzuka': 5807,
    'Bahrain': 5412,
    'Jeddah': 6174,
    'Miami': 5412,
    'Imola': 4909,
    'Monaco': 3337,
    'Spagna': 4657,
    'Montreal': 4361,
    'Austria': 4318,
    'Silverstone': 5891,
    'Spa': 7004,
    'Hungaroring': 4381,
    'Zandvoort': 4259,
    'Monza': 5793,
    'Baku': 6003,
    'Singapore': 4940,
    'Austin': 5513,
    'Mexico City': 4304,
    'Interlagos': 4309,
    'Las Vegas': 6201,
    'Qatar': 5419,
    'Abu Dhabi': 5281,
    'Le Castellet': 5842,
    'Sochi': 5848,
    'Portimao': 4653,
    'Istanbul': 5338,
    'Nurburgring': 5148,
    'Hockenheim': 4574,
    
    'Adelaide': 3780,
    'Phoenix': 3770,
    'Estoril': 4415,
    'Magny-Cours': 4411,
    'Indianapolis': 4180,
    'Sepang': 5543,
    'Valencia': 4005,
    'Yeongam': 5603,
    'Jerez': 4423,
    'Kyalami': 4300,
    'Estoril': 4415,
    'Donington Park': 4023,
    'Fuji': 4571
}.get(race, 0) 

# Get the laps data for the target driver
driver_laps = session.laps.pick_driver(target_driver)

# Prepare the data for analysis
tire_data = []
current_stint = None
cumulative_tire_distance = 0

for lap in driver_laps.iterlaps():
    lap_number = lap[1]['LapNumber']
    lap_time = lap[1]['LapTime']
    compound = lap[1]['Compound']
    tire_life = lap[1]['TyreLife']
    fresh_tire = lap[1]['FreshTyre']
    stint = lap[1]['Stint']
    
    # Calculate meters driven on these tires
    if stint != current_stint:
        # New tire stint begins
        cumulative_tire_distance = 0
        current_stint = stint
    
    # Add this lap's distance to the cumulative tire distance
    cumulative_tire_distance += circuit_length
    
    tire_data.append({
        'Driver': target_driver,
        'LapNumber': lap_number,
        'Stint': stint,
        'TireLife': tire_life,
        'TireMeters': cumulative_tire_distance,
        'Compound': compound,
        'IsFreshTire': fresh_tire,
        'LapTime': lap_time,
    })

# Create DataFrame
tire_df = pd.DataFrame(tire_data)

# Save to CSV
tire_df.to_csv(f'{target_driver}_{race}_{year}_tire_data.csv', index=False)

# Display some information
print(f"Location: {session.event.Location}")
print(f"Length of the circuit: {circuit_length:.0f} meters")
print("\nTire data summary:")
print(tire_df[['LapNumber', 'Compound', 'TireLife', 'TireMeters', 'IsFreshTire']].head())

core           INFO 	Loading data for Monaco Grand Prix - Race [v3.4.5]
req            INFO 	No cached data found for session_info. Loading data...
_api           INFO 	Fetching session info data...
req            INFO 	Data has been written to cache!
req            INFO 	No cached data found for driver_info. Loading data...
_api           INFO 	Fetching driver list...
req            INFO 	Data has been written to cache!
req            INFO 	No cached data found for session_status_data. Loading data...
_api           INFO 	Fetching session status data...
req            INFO 	Data has been written to cache!
req            INFO 	No cached data found for lap_count. Loading data...
_api           INFO 	Fetching lap count data...
req            INFO 	Data has been written to cache!
req            INFO 	No cached data found for track_status_data. Loading data...
_api           INFO 	Fetching track status data...
req            INFO 	Data has been written to cache!
req            INFO 	No cac

Location: Monaco
Length of the circuit: 3337 meters

Tire data summary:
   LapNumber Compound  TireLife  TireMeters  IsFreshTire
0        1.0     HARD       1.0        3337         True
1        2.0     HARD       2.0        6674         True
2        3.0     HARD       3.0       10011         True
3        4.0     HARD       4.0       13348         True
4        5.0     HARD       5.0       16685         True


c:\Users\dilau\Desktop\TeamProjF1\FASTF1\.venv\Lib\site-packages\fastf1\core.py:3022: FutureWarning: pick_driver is deprecated and will be removed in a future release. Use pick_drivers instead.
  warnings.warn(("pick_driver is deprecated and will be removed"
